# Step 1 - Preparation of the flat file

# Load the libraries



In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline
import string
from IPython.display import display
import datetime


In [4]:
# set the max columns to none
pd.set_option('display.max_columns', None)

pd.set_option('display.max_colwidth', None)

# Show all columns and set a wider display width
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

In [5]:
# to reset all this options

#pd.reset_option('display.max_rows')
#pd.reset_option('display.max_columns')
# pd.reset_option('display.max_colwidth')

In [6]:
from IPython.display import HTML
# This CSS will make all output cells scrollable horizontally.
HTML("""
<style>
div.output_scroll {
    overflow-x: auto;
}
</style>
""")

# Data preparation

Upload the data


In [9]:
df_prep_spotify_songs = pd.read_csv(r"C:\Users\kagan\Dropbox\קורס DS בר-אילן\שיעורים\Projects\Project 3 - ML preparation and model\Spotify_data\spotify_songs.csv"
                                  ) # work TO CANCEL THE INDEX COLUMN WRITE index_col=0


In [10]:
df_prep_spotify_songs.head()

,track_id,track_name,track_artist,track_popularity,track_album_id,track_album_name,track_album_release_date,playlist_name,playlist_id,playlist_genre,playlist_subgenre,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
0,6f807x0ima9a1j3VPbc7VN,I Don't Care (with Justin Bieber) - Loud Luxury Remix,Ed Sheeran,66,2oCs0DGTsRO98Gh5ZSl2Cx,I Don't Care (with Justin Bieber) [Loud Luxury Remix],2019-06-14,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,dance pop,0.748,0.916,6,-2.634,1,0.0583,0.1020,0.000000,0.0653,0.518,122.036,194754
1,0r7CVbZTWZgbTCYdfa2P31,Memories - Dillon Francis Remix,Maroon 5,67,63rPSO264uRjW1X5E6cWv6,Memories (Dillon Francis Remix),2019-12-13,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,dance pop,0.726,0.815,11,-4.969,1,0.0373,0.0724,0.004210,0.3570,0.693,99.972,162600
2,1z1Hg7Vb0AhHDiEmnDE79l,All the Time - Don Diablo Remix,Zara Larsson,70,1HoSmj2eLcsrR0vE9gThr4,All the Time (Don Diablo Remix),2019-07-05,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,dance pop,0.675,0.931,1,-3.432,0,0.0742,0.0794,0.000023,0.1100,0.613,124.008,176616
3,75FpbthrwQmzHlBJLuGdC7,Call You Mine - Keanu Silva Remix,The Chainsmokers,60,1nqYsOef1yKKuGOVchbsk6,Call You Mine - The Remixes,2019-07-19,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,dance pop,0.718,0.930,7,-3.778,1,0.1020,0.0287,0.000009,0.2040,0.277,121.956,169093
4,1e8PAfcKUYoKkxPhrHqw4x,Someone You Loved - Future Humans Remix,Lewis Capaldi,69,7m7vv9wlQ4i0LFuJiE2zsQ,Someone You Loved (Future Humans Remix),2019-03-05,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,dance pop,0.650,0.833,1,-4.672,1,0.0359,0.0803,0.000000,0.0833,0.725,123.976,189052


In [11]:
def analyze_columns(df, include_duplicates=False):
    """
    Returns a summary DataFrame with:
      - # NA: Count of explicit NA values (e.g., np.nan) per column.
      - % NA: Percentage of explicit NA values per column.
      - # Missing (Total): Count of all missing values (NA + None) per column.
      - % Missing (Total): Percentage of all missing values per column.
      - # Unique: Number of unique values per column.
      - % Unique: Percentage of unique (non-missing) values per column.
      - Min: Minimum value in the column.
      - Max: Maximum value in the column.
      - Most Frequent: The most frequently occurring value in the column.
      - Mode: Frequency count of the most frequent value.
      - Mean: Arithmetic mean (for numeric columns; NaN for others).
      
      Optionally, if include_duplicates=True, adds:
          - # Duplicates: Count of duplicate non-missing values.
          - % Duplicates: Percentage of duplicate non-missing values.
      
      Also appends an average row at the bottom (for numeric metrics only; new columns set to NaN).
    """
    import numpy as np
    import pandas as pd
    
    total_rows = len(df)
    
    # Helper functions for safe calculations
    def safe_min(col):
        try:
            if pd.api.types.is_numeric_dtype(col):
                return col.min()
            elif pd.api.types.is_string_dtype(col):
                return col.dropna().astype(str).min()
            else:
                return col.dropna().min()
        except Exception:
            return np.nan

    def safe_max(col):
        try:
            if pd.api.types.is_numeric_dtype(col):
                return col.max()
            elif pd.api.types.is_string_dtype(col):
                return col.dropna().astype(str).max()
            else:
                return col.dropna().max()
        except Exception:
            return np.nan

    def safe_most_frequent(col):
        try:
            mode_vals = col.mode()
            if not mode_vals.empty:
                return mode_vals.iloc[0]
            else:
                return np.nan
        except Exception:
            return np.nan

    def safe_mode_count(col):
        try:
            mode_vals = col.mode()
            if not mode_vals.empty:
                mode_val = mode_vals.iloc[0]
                return (col == mode_val).sum()
            else:
                return np.nan
        except Exception:
            return np.nan

    def safe_mean(col):
        try:
            if pd.api.types.is_numeric_dtype(col):
                return col.mean()
            else:
                return np.nan
        except Exception:
            return np.nan

    # Calculate explicit NA (np.nan) and total missing values
    na_counts = df.isna().sum()
    missing_counts = df.isnull().sum()
    
    # Create initial summary with missing and unique metrics
    summary = pd.DataFrame({
        'Column': df.columns,
        '# NA': na_counts,
        '% NA': na_counts / total_rows * 100,
        '# Missing (Total)': missing_counts,
        '% Missing (Total)': missing_counts / total_rows * 100,
        '# Unique': df.nunique(),
        '% Unique': df.nunique() / total_rows * 100,
    })
    
    if include_duplicates:
        # Compute count of non-missing values per column
        valid_counts = total_rows - missing_counts
        # Calculate duplicates on non-missing data: duplicates = non-missing count - unique count
        summary['# Duplicates'] = valid_counts - df.nunique()
        # Calculate percentage duplicates relative to the non-missing count
        summary['% Duplicates'] = summary['# Duplicates'] / valid_counts * 100
    
    # Add additional summary statistics using the safe helper functions
    summary['Min'] = df.apply(lambda col: safe_min(col))
    summary['Max'] = df.apply(lambda col: safe_max(col))
    summary['Most Frequent'] = df.apply(lambda col: safe_most_frequent(col))
    summary['Mode'] = df.apply(lambda col: safe_mode_count(col))
    summary['Mean'] = df.apply(lambda col: safe_mean(col))
    
    # Compute the average for numeric columns from the original summary (if applicable)
    avg_row = {
        'Column': '**Average**',
        '# NA': summary['# NA'].mean(),
        '% NA': summary['% NA'].mean(),
        '# Missing (Total)': summary['# Missing (Total)'].mean(),
        '% Missing (Total)': summary['% Missing (Total)'].mean(),
        '# Unique': summary['# Unique'].mean(),
        '% Unique': summary['% Unique'].mean(),
        # For the added columns, averaging doesn't make sense; set as NaN.
        'Min': np.nan,
        'Max': np.nan,
        'Most Frequent': np.nan,
        'Mode': np.nan,
        'Mean': np.nan,
    }
    
    if include_duplicates:
        avg_row['# Duplicates'] = summary['# Duplicates'].mean()
        avg_row['% Duplicates'] = summary['% Duplicates'].mean()
    
    avg_df = pd.DataFrame([avg_row])
    
    # Append the average row
    summary = pd.concat([summary, avg_df], ignore_index=True)
    
    return summary

In [12]:
# Usage of the ana:
column_summary = analyze_columns(df_prep_spotify_songs, include_duplicates=True)

display(column_summary)

,Column,# NA,% NA,# Missing (Total),% Missing (Total),# Unique,% Unique,# Duplicates,% Duplicates,Min,Max,Most Frequent,Mode,Mean
0,track_id,0.000000,0.000000,0.000000,0.000000,28356.000000,86.364329,4477.000000,13.635671,0017A6SJgTbfQVU2EtsPNo,7zzZmpw8L66ZPjH1M6qmOs,7BKLCZ1jbUBVqRi2FVlTVw,10.0,NaN
1,track_name,5.000000,0.015229,5.000000,0.015229,23449.000000,71.418999,9379.000000,28.570123,"""I TRIED FOR YEARS... NOBODY LISTENED""",하드캐리 Hard Carry,Poison,22.0,NaN
2,track_artist,5.000000,0.015229,5.000000,0.015229,10692.000000,32.564798,22136.000000,67.430242,!!!,香取慎吾,Martin Garrix,161.0,NaN
3,track_popularity,0.000000,0.000000,0.000000,0.000000,101.000000,0.307617,32732.000000,99.692383,0,100,0,2703.0,42.477081
4,track_album_id,0.000000,0.000000,0.000000,0.000000,22545.000000,68.665672,10288.000000,31.334328,000YOrgQoB5IiiH95Yb8vY,7zygyMUltFYOvHoT3NOTsj,5L1xcowSxwzFUSJzvyMp48,42.0,NaN
5,track_album_name,5.000000,0.015229,5.000000,0.015229,19743.000000,60.131575,13085.000000,39.859266,!,화양연화 The Most Beautiful Moment In Life: Young Forever,Greatest Hits,139.0,NaN
6,track_album_release_date,0.000000,0.000000,0.000000,0.000000,4530.000000,13.797094,28303.000000,86.202906,1957-01-01,2020-01-29,2020-01-10,270.0,NaN
7,playlist_name,0.000000,0.000000,0.000000,0.000000,449.000000,1.367527,32384.000000,98.632473,"""Permanent Wave""",🤩🤪Post Teen Pop🤪🤩,Indie Poptimism,308.0,NaN
8,playlist_id,0.000000,0.000000,0.000000,0.000000,471.000000,1.434532,32362.000000,98.565468,0275i1VNfBnsNbPl0QIBpG,7xWuNevFBmwnFEg6wzdCc7,4JkkvMpVl4lSioqQjeAL0q,247.0,NaN
9,playlist_genre,0.000000,0.000000,0.000000,0.000000,6.000000,0.018274,32827.000000,99.981726,edm,rock,edm,6043.0,NaN


In [13]:
# let's see if dataset contains missing_values
df_prep_spotify_songs.isnull().sum()

track_id                    0
track_name                  5
track_artist                5
track_popularity            0
track_album_id              0
track_album_name            5
track_album_release_date    0
playlist_name               0
playlist_id                 0
playlist_genre              0
playlist_subgenre           0
danceability                0
energy                      0
key                         0
loudness                    0
mode                        0
speechiness                 0
acousticness                0
instrumentalness            0
liveness                    0
valence                     0
tempo                       0
duration_ms                 0
dtype: int64

In [14]:
df_prep_spotify_songs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32833 entries, 0 to 32832
Data columns (total 23 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   track_id                  32833 non-null  object 
 1   track_name                32828 non-null  object 
 2   track_artist              32828 non-null  object 
 3   track_popularity          32833 non-null  int64  
 4   track_album_id            32833 non-null  object 
 5   track_album_name          32828 non-null  object 
 6   track_album_release_date  32833 non-null  object 
 7   playlist_name             32833 non-null  object 
 8   playlist_id               32833 non-null  object 
 9   playlist_genre            32833 non-null  object 
 10  playlist_subgenre         32833 non-null  object 
 11  danceability              32833 non-null  float64
 12  energy                    32833 non-null  float64
 13  key                       32833 non-null  int64  
 14  loudne

In [15]:
print(df_prep_spotify_songs.columns)

Index(['track_id', 'track_name', 'track_artist', 'track_popularity', 'track_album_id', 'track_album_name', 'track_album_release_date', 'playlist_name', 'playlist_id', 'playlist_genre', 'playlist_subgenre', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms'], dtype='object')


In [16]:
df_prep_spotify_songs.shape # (32833, 23)

(32833, 23)

# Prepare the date time for album releate date


In [18]:
# check the nulls
print(df_prep_spotify_songs['track_album_release_date'].isnull().sum()) # 0
print(df_prep_spotify_songs['track_album_release_date'].dtype) # object

0
object


In [19]:
# Filter rows where only the year (4 characters) is present in 'track_album_release_date'
only_year_rows = df_prep_spotify_songs[df_prep_spotify_songs['track_album_release_date'].str.match(r'^\d{4}$')]
only_year_rows.head()

,track_id,track_name,track_artist,track_popularity,track_album_id,track_album_name,track_album_release_date,playlist_name,playlist_id,playlist_genre,playlist_subgenre,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
151,2QtJA4gbwe1AcanB2p21aP,Stay The Night - Featuring Hayley Williams Of Paramore,Zedd,61,4xak0izXQHOMV9Wu1mDAAy,Clarity (Deluxe),2012,Dance Pop,37i9dQZF1DWZQaaqNMbbXa,pop,dance pop,0.596,0.738,8,-3.109,1,0.0411,0.10900,0.000000,0.0947,0.461,127.961,217347
749,77j9HOrxUpWcjwra9jgvio,We like to Party! (Six Flags),Vengaboys,51,0R44mmTKUE1XCuGVVrPfTZ,We like to Party! (The Vengabus),1998,90s Dance Hits,50cXvxgMGrvoUeLhkzCyIO,pop,dance pop,0.785,0.905,8,-7.929,1,0.0449,0.00803,0.859000,0.0460,0.969,136.018,224227
750,6rQ9qJPvInEqSzlh89LsPJ,Coco Jamboo - Radio Version,Mr. President,18,2t8Bc4qBseswbwOoHGHK8d,We See The Same Sun,1996,90s Dance Hits,50cXvxgMGrvoUeLhkzCyIO,pop,dance pop,0.832,0.838,1,-5.414,0,0.0421,0.07910,0.000000,0.1520,0.965,100.019,218733
751,5FgtdSf7I5lClThz2ptWvl,Blue (Da Ba Dee),Eiffel 65,16,65DySolRDG1LNSvRXcWQWN,Europop,1999,90s Dance Hits,50cXvxgMGrvoUeLhkzCyIO,pop,dance pop,0.822,0.969,7,-11.471,0,0.0582,0.25900,0.000162,0.3900,0.765,128.007,283748
753,2ahnofp2LbBWDXcJbMaSTu,What Is Love,Haddaway,70,0qTVjeVP70YvQG4GctetOf,What is Love - Remix,1993,90s Dance Hits,50cXvxgMGrvoUeLhkzCyIO,pop,dance pop,0.683,0.772,7,-7.907,0,0.0311,0.02220,0.014900,0.2030,0.737,123.871,270373


This following step was done to ensure we will not get NA when we convert to date time type because if we do not do this step,
we will get NA value each time we have only the Year of the release album date, as some rows contain the full date and some only the Year of the album release. This function also converts to Year and month of the release date. 

In [21]:

def process_mixed_dates(df, date_column='track_album_release_date'):
    """
    Handles mixed date formats (year-only and full dates) in a DataFrame.

    Args:
        df (pd.DataFrame): The input DataFrame.
        date_column (str): The name of the date column.

    Returns:
        pd.DataFrame: The DataFrame with the date column converted to datetime, or kept as string if parsing fails.
    """

    def parse_date(date_str):
        try:
            return pd.to_datetime(date_str)
        except (ValueError, TypeError): #added TypeError
            try:
                year = int(date_str)
                return pd.to_datetime(f'{year}-01-01') #assign the first day of the year.
            except (ValueError, TypeError): #added TypeError
                return date_str #keep the original string.

    # Apply the parsing and convert to datetime, or keep as string.
    df[date_column] = df[date_column].apply(parse_date)

    # Convert to datetime, coercing if needed.
    df[date_column] = pd.to_datetime(df[date_column], errors='coerce')

    # Extract year and month (if needed)
    df['track_album_release_year'] = df[date_column].dt.year
    df['track_album_release_month'] = df[date_column].dt.month


    return df

In [22]:
df_prep_spotify_songs1 = process_mixed_dates(df_prep_spotify_songs)

In [23]:
print(df_prep_spotify_songs1['track_album_release_date'].isnull().sum()) # 0 
print(df_prep_spotify_songs1['track_album_release_date'].dtype) # datetime64[ns]

0
datetime64[ns]


In [24]:
# Get the current year
current_year = datetime.datetime.now().year

# Calculate the song's age from release
df_prep_spotify_songs1['song_age'] = current_year - df_prep_spotify_songs1['track_album_release_year']


In [25]:
# Drop the 'track_album_release_date' column
df_prep_spotify_songs1.drop('track_album_release_date', axis=1, inplace=True)


In [26]:
# Display the first few rows to verify the changes
print(df_prep_spotify_songs1[[ 'track_album_release_year','track_album_release_month', 'song_age']].head())

   track_album_release_year  track_album_release_month  song_age
0                      2019                          6         6
1                      2019                         12         6
2                      2019                          7         6
3                      2019                          7         6
4                      2019                          3         6


In [27]:
df_prep_spotify_songs1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32833 entries, 0 to 32832
Data columns (total 25 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   track_id                   32833 non-null  object 
 1   track_name                 32828 non-null  object 
 2   track_artist               32828 non-null  object 
 3   track_popularity           32833 non-null  int64  
 4   track_album_id             32833 non-null  object 
 5   track_album_name           32828 non-null  object 
 6   playlist_name              32833 non-null  object 
 7   playlist_id                32833 non-null  object 
 8   playlist_genre             32833 non-null  object 
 9   playlist_subgenre          32833 non-null  object 
 10  danceability               32833 non-null  float64
 11  energy                     32833 non-null  float64
 12  key                        32833 non-null  int64  
 13  loudness                   32833 non-null  flo

# Convert the object columns to string for consistency

In [29]:
def convert_object_columns_to_strings(df):
    """
    Converts all object type columns in a Pandas DataFrame to string type.

    Args:
        df (pd.DataFrame): The input DataFrame.

    Returns:
        pd.DataFrame: The DataFrame with object columns converted to strings.
    """
    object_cols = df.select_dtypes(include=['object']).columns
    df[object_cols] = df[object_cols].astype("string")
    return df

def check_string_columns(df):
    """
    Checks which columns in a Pandas DataFrame are of string type.

    Args:
        df (pd.DataFrame): The input DataFrame.

    Returns:
        list: A list of column names that are strings.
    """
    string_cols = df.select_dtypes(include=['object']).columns.tolist() #object type in pandas is string
    return string_cols

# Convert object columns to strings
df_prep_spotify_song2 = convert_object_columns_to_strings(df_prep_spotify_songs1)

In [30]:
df_prep_spotify_song2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32833 entries, 0 to 32832
Data columns (total 25 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   track_id                   32833 non-null  string 
 1   track_name                 32828 non-null  string 
 2   track_artist               32828 non-null  string 
 3   track_popularity           32833 non-null  int64  
 4   track_album_id             32833 non-null  string 
 5   track_album_name           32828 non-null  string 
 6   playlist_name              32833 non-null  string 
 7   playlist_id                32833 non-null  string 
 8   playlist_genre             32833 non-null  string 
 9   playlist_subgenre          32833 non-null  string 
 10  danceability               32833 non-null  float64
 11  energy                     32833 non-null  float64
 12  key                        32833 non-null  int64  
 13  loudness                   32833 non-null  flo

In [31]:
# Check the data type of the column
print(f"Type of df_prep_spotify_song1: {type(df_prep_spotify_song2)}") 

Type of df_prep_spotify_song1: <class 'pandas.core.frame.DataFrame'>


In [32]:
df_prep_spotify_song2.head()


,track_id,track_name,track_artist,track_popularity,track_album_id,track_album_name,playlist_name,playlist_id,playlist_genre,playlist_subgenre,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,track_album_release_year,track_album_release_month,song_age
0,6f807x0ima9a1j3VPbc7VN,I Don't Care (with Justin Bieber) - Loud Luxury Remix,Ed Sheeran,66,2oCs0DGTsRO98Gh5ZSl2Cx,I Don't Care (with Justin Bieber) [Loud Luxury Remix],Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,dance pop,0.748,0.916,6,-2.634,1,0.0583,0.1020,0.000000,0.0653,0.518,122.036,194754,2019,6,6
1,0r7CVbZTWZgbTCYdfa2P31,Memories - Dillon Francis Remix,Maroon 5,67,63rPSO264uRjW1X5E6cWv6,Memories (Dillon Francis Remix),Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,dance pop,0.726,0.815,11,-4.969,1,0.0373,0.0724,0.004210,0.3570,0.693,99.972,162600,2019,12,6
2,1z1Hg7Vb0AhHDiEmnDE79l,All the Time - Don Diablo Remix,Zara Larsson,70,1HoSmj2eLcsrR0vE9gThr4,All the Time (Don Diablo Remix),Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,dance pop,0.675,0.931,1,-3.432,0,0.0742,0.0794,0.000023,0.1100,0.613,124.008,176616,2019,7,6
3,75FpbthrwQmzHlBJLuGdC7,Call You Mine - Keanu Silva Remix,The Chainsmokers,60,1nqYsOef1yKKuGOVchbsk6,Call You Mine - The Remixes,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,dance pop,0.718,0.930,7,-3.778,1,0.1020,0.0287,0.000009,0.2040,0.277,121.956,169093,2019,7,6
4,1e8PAfcKUYoKkxPhrHqw4x,Someone You Loved - Future Humans Remix,Lewis Capaldi,69,7m7vv9wlQ4i0LFuJiE2zsQ,Someone You Loved (Future Humans Remix),Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,dance pop,0.650,0.833,1,-4.672,1,0.0359,0.0803,0.000000,0.0833,0.725,123.976,189052,2019,3,6


In [33]:
df_prep_spotify_song2.tail()

,track_id,track_name,track_artist,track_popularity,track_album_id,track_album_name,playlist_name,playlist_id,playlist_genre,playlist_subgenre,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,track_album_release_year,track_album_release_month,song_age
32828,7bxnKAamR3snQ1VGLuVfC1,City Of Lights - Official Radio Edit,Lush & Simon,42,2azRoBBWEEEYhqV6sb7JrT,City Of Lights (Vocal Mix),♥ EDM LOVE 2020,6jI1gFr6ANFtT8MmTvA2Ux,edm,progressive electro house,0.428,0.922,2,-1.814,1,0.0936,0.076600,0.000000,0.0668,0.2100,128.170,204375,2014,4,11
32829,5Aevni09Em4575077nkWHz,Closer - Sultan & Ned Shepard Remix,Tegan and Sara,20,6kD6KLxj7s8eCE3ABvAyf5,Closer Remixed,♥ EDM LOVE 2020,6jI1gFr6ANFtT8MmTvA2Ux,edm,progressive electro house,0.522,0.786,0,-4.462,1,0.0420,0.001710,0.004270,0.3750,0.4000,128.041,353120,2013,3,12
32830,7ImMqPP3Q1yfUHvsdn7wEo,Sweet Surrender - Radio Edit,Starkillers,14,0ltWNSY9JgxoIZO4VzuCa6,Sweet Surrender (Radio Edit),♥ EDM LOVE 2020,6jI1gFr6ANFtT8MmTvA2Ux,edm,progressive electro house,0.529,0.821,6,-4.899,0,0.0481,0.108000,0.000001,0.1500,0.4360,127.989,210112,2014,4,11
32831,2m69mhnfQ1Oq6lGtXuYhgX,Only For You - Maor Levi Remix,Mat Zo,15,1fGrOkHnHJcStl14zNx8Jy,Only For You (Remixes),♥ EDM LOVE 2020,6jI1gFr6ANFtT8MmTvA2Ux,edm,progressive electro house,0.626,0.888,2,-3.361,1,0.1090,0.007920,0.127000,0.3430,0.3080,128.008,367432,2014,1,11
32832,29zWqhca3zt5NsckZqDf6c,Typhoon - Original Mix,Julian Calor,27,0X3mUOm6MhxR7PzxG95rAo,Typhoon/Storm,♥ EDM LOVE 2020,6jI1gFr6ANFtT8MmTvA2Ux,edm,progressive electro house,0.603,0.884,5,-4.571,0,0.0385,0.000133,0.341000,0.7420,0.0894,127.984,337500,2014,3,11


# Convert the string columns lower case

In [35]:
# Normalize selected text columns convert to lowercase and Remove any leading or trailing whitespace from the string by str.strip()

df_prep_spotify_song2['track_name'] = df_prep_spotify_song2['track_name'].str.strip().str.lower()
df_prep_spotify_song2['track_artist'] = df_prep_spotify_song2['track_artist'].str.strip().str.lower()
df_prep_spotify_song2['track_album_name'] = df_prep_spotify_song2['track_album_name'].str.strip().str.lower()
df_prep_spotify_song2['playlist_name'] = df_prep_spotify_song2['playlist_name'].str.strip().str.lower()
df_prep_spotify_song2['playlist_genre'] = df_prep_spotify_song2['playlist_genre'].str.strip().str.lower()

In [36]:
display(df_prep_spotify_song2)

,track_id,track_name,track_artist,track_popularity,track_album_id,track_album_name,playlist_name,playlist_id,playlist_genre,playlist_subgenre,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,track_album_release_year,track_album_release_month,song_age
0,6f807x0ima9a1j3VPbc7VN,i don't care (with justin bieber) - loud luxury remix,ed sheeran,66,2oCs0DGTsRO98Gh5ZSl2Cx,i don't care (with justin bieber) [loud luxury remix],pop remix,37i9dQZF1DXcZDD7cfEKhW,pop,dance pop,0.748,0.916,6,-2.634,1,0.0583,0.102000,0.000000,0.0653,0.5180,122.036,194754,2019,6,6
1,0r7CVbZTWZgbTCYdfa2P31,memories - dillon francis remix,maroon 5,67,63rPSO264uRjW1X5E6cWv6,memories (dillon francis remix),pop remix,37i9dQZF1DXcZDD7cfEKhW,pop,dance pop,0.726,0.815,11,-4.969,1,0.0373,0.072400,0.004210,0.3570,0.6930,99.972,162600,2019,12,6
2,1z1Hg7Vb0AhHDiEmnDE79l,all the time - don diablo remix,zara larsson,70,1HoSmj2eLcsrR0vE9gThr4,all the time (don diablo remix),pop remix,37i9dQZF1DXcZDD7cfEKhW,pop,dance pop,0.675,0.931,1,-3.432,0,0.0742,0.079400,0.000023,0.1100,0.6130,124.008,176616,2019,7,6
3,75FpbthrwQmzHlBJLuGdC7,call you mine - keanu silva remix,the chainsmokers,60,1nqYsOef1yKKuGOVchbsk6,call you mine - the remixes,pop remix,37i9dQZF1DXcZDD7cfEKhW,pop,dance pop,0.718,0.930,7,-3.778,1,0.1020,0.028700,0.000009,0.2040,0.2770,121.956,169093,2019,7,6
4,1e8PAfcKUYoKkxPhrHqw4x,someone you loved - future humans remix,lewis capaldi,69,7m7vv9wlQ4i0LFuJiE2zsQ,someone you loved (future humans remix),pop remix,37i9dQZF1DXcZDD7cfEKhW,pop,dance pop,0.650,0.833,1,-4.672,1,0.0359,0.080300,0.000000,0.0833,0.7250,123.976,189052,2019,3,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32828,7bxnKAamR3snQ1VGLuVfC1,city of lights - official radio edit,lush & simon,42,2azRoBBWEEEYhqV6sb7JrT,city of lights (vocal mix),♥ edm love 2020,6jI1gFr6ANFtT8MmTvA2Ux,edm,progressive electro house,0.428,0.922,2,-1.814,1,0.0936,0.076600,0.000000,0.0668,0.2100,128.170,204375,2014,4,11
32829,5Aevni09Em4575077nkWHz,closer - sultan & ned shepard remix,tegan and sara,20,6kD6KLxj7s8eCE3ABvAyf5,closer remixed,♥ edm love 2020,6jI1gFr6ANFtT8MmTvA2Ux,edm,progressive electro house,0.522,0.786,0,-4.462,1,0.0420,0.001710,0.004270,0.3750,0.4000,128.041,353120,2013,3,12
32830,7ImMqPP3Q1yfUHvsdn7wEo,sweet surrender - radio edit,starkillers,14,0ltWNSY9JgxoIZO4VzuCa6,sweet surrender (radio edit),♥ edm love 2020,6jI1gFr6ANFtT8MmTvA2Ux,edm,progressive electro house,0.529,0.821,6,-4.899,0,0.0481,0.108000,0.000001,0.1500,0.4360,127.989,210112,2014,4,11
32831,2m69mhnfQ1Oq6lGtXuYhgX,only for you - maor levi remix,mat zo,15,1fGrOkHnHJcStl14zNx8Jy,only for you (remixes),♥ edm love 2020,6jI1gFr6ANFtT8MmTvA2Ux,edm,progressive electro house,0.626,0.888,2,-3.361,1,0.1090,0.007920,0.127000,0.3430,0.3080,128.008,367432,2014,1,11


In [37]:
display(analyze_columns(df_prep_spotify_song2, include_duplicates=True))

,Column,# NA,% NA,# Missing (Total),% Missing (Total),# Unique,% Unique,# Duplicates,% Duplicates,Min,Max,Most Frequent,Mode,Mean
0,track_id,0.0,0.000000,0.0,0.000000,28356.00,86.364329,4477.00,13.635671,0017A6SJgTbfQVU2EtsPNo,7zzZmpw8L66ZPjH1M6qmOs,7BKLCZ1jbUBVqRi2FVlTVw,10.0,NaN
1,track_name,5.0,0.015229,5.0,0.015229,23095.00,70.340816,9733.00,29.648471,"""i tried for years... nobody listened""",하드캐리 hard carry,poison,22.0,NaN
2,track_artist,5.0,0.015229,5.0,0.015229,10685.00,32.543478,22143.00,67.451566,!!!,香取慎吾,martin garrix,161.0,NaN
3,track_popularity,0.0,0.000000,0.0,0.000000,101.00,0.307617,32732.00,99.692383,0,100,0,2703.0,42.477081
4,track_album_id,0.0,0.000000,0.0,0.000000,22545.00,68.665672,10288.00,31.334328,000YOrgQoB5IiiH95Yb8vY,7zygyMUltFYOvHoT3NOTsj,5L1xcowSxwzFUSJzvyMp48,42.0,NaN
5,track_album_name,5.0,0.015229,5.0,0.015229,19533.00,59.491975,13295.00,40.498964,!,화양연화 the most beautiful moment in life: young forever,greatest hits,139.0,NaN
6,playlist_name,0.0,0.000000,0.0,0.000000,433.00,1.318795,32400.00,98.681205,"""permanent wave""",🤩🤪post teen pop🤪🤩,permanent wave,470.0,NaN
7,playlist_id,0.0,0.000000,0.0,0.000000,471.00,1.434532,32362.00,98.565468,0275i1VNfBnsNbPl0QIBpG,7xWuNevFBmwnFEg6wzdCc7,4JkkvMpVl4lSioqQjeAL0q,247.0,NaN
8,playlist_genre,0.0,0.000000,0.0,0.000000,6.00,0.018274,32827.00,99.981726,edm,rock,edm,6043.0,NaN
9,playlist_subgenre,0.0,0.000000,0.0,0.000000,24.00,0.073097,32809.00,99.926903,album rock,urban contemporary,progressive electro house,1809.0,NaN


# Data Validation Steps:

Duration Filter: Removed songs under 30 seconds as they probably were some of the examples.

Tempo Threshold Analysis: Tracks with a tempo below 20 BPM were excluded from the dataset, as such values fall significantly outside the standard range of typical musical compositions. Given that the dataset's mean tempo is 120 BPM, this threshold was established to ensure the selection of musically significant and analytically relevant tracks for further study.

In [39]:
# Filter Out Songs with Low Tempo or Very Short Duration

# Apply filtering conditions on df_prep_spotify_song6:
# - Keep only rows where tempo is at least 20 and duration_ms is at least 30,000.
df_prep_spotify_song3 = df_prep_spotify_song2[
    (df_prep_spotify_song2['tempo'] >= 20) &
    (df_prep_spotify_song2['duration_ms'] >= 30000)
].copy()

# Print the number of rows before filtering.
print("Number of rows before filtering:", len(df_prep_spotify_song2))
# Print the number of rows after filtering and the number of rows removed.
print("Number of rows after filtering:", len(df_prep_spotify_song3))
print("Number of deleted rows:", len(df_prep_spotify_song2) - len(df_prep_spotify_song3))

Number of rows before filtering: 32833
Number of rows after filtering: 32831
Number of deleted rows: 2


# Dropping NA rows

In [41]:

df_prep_spotify_song3 = df_prep_spotify_song3.dropna(subset=['track_name', 'track_artist'])

# Print the length of the data after drop na
print("Number of rows after dropping the NA:", len(df_prep_spotify_song3))


Number of rows after dropping the NA: 32826


# Delete duplicates

In [43]:
# Define a helper function to get a Series's mode (most frequent value) of a Series.
# Becuase we want precisely delete the duplicates as if a few duplicated rows have 
# different gunre or popularity we will choose those of the most frequent

def get_mode(series):
    """
    Returns the most frequent (mode) value in a Series.
    If multiple modes exist, returns the first one.
    This function is used to select a representative value for duplicate entries.
    """
    mode_vals = series.mode()
    if not mode_vals.empty:
        return mode_vals.iloc[0]
    else:
        # If mode cannot be computed, default to the first element.
        return series.iloc[0]


# Group by 'track_id' to handle duplicates:
# For each group (same track_id), we decide which row to keep:
# - For 'track_popularity' and 'playlist_genre', we use the mode (most frequent value)
#   because duplicate rows might have different values for these columns.
# - For other columns, we take the first occurrence.

df_prep_spotify_song4 = df_prep_spotify_song3.groupby('track_id').agg({
    'track_name': 'first',
    'track_artist': 'first',
    'track_popularity': get_mode,       # Mode of popularity among duplicates
    'track_album_id': 'first',
    'track_album_name': 'first',
    'playlist_name': 'first',           # You could also use mode here if desired
    'playlist_id': 'first',
    'playlist_genre': get_mode,         # Mode (most frequent genre)
    'playlist_subgenre': 'first',
    'danceability': 'first',
    'energy': 'first',
    'key': 'first',
    'loudness': 'first',
    'mode': 'first',
    'speechiness': 'first',
    'acousticness': 'first',
    'instrumentalness': 'first',
    'liveness': 'first',
    'valence': 'first',
    'tempo': 'first',
    'duration_ms': 'first',
    'track_album_release_year': 'first',
    "track_album_release_month": 'first',
    'song_age': 'first'
}).reset_index()

# Print the initial number of rows before handling duplicates
print("Number of rows before duplicate handling:", len(df_prep_spotify_song3))

# Check how many rows remain after duplicate handling.
print("Number of rows after duplicate handling:", len(df_prep_spotify_song4))
print("deleted duplicated rows:", len(df_prep_spotify_song3)-len(df_prep_spotify_song4))

Number of rows before duplicate handling: 32826
Number of rows after duplicate handling: 28350
deleted duplicated rows: 4476


Identify the rest of the duplicates by the track name, track artist, and track popularity. 


In [45]:
df_prep_spotify_song4[df_prep_spotify_song4.duplicated(subset=['track_name', 'track_artist', 'track_popularity'],
                                                       keep=False)].groupby(['track_name', 'track_artist', 'track_popularity']).size().reset_index(name='count')

,track_name,track_artist,track_popularity,count
0,2 of amerikaz most wanted,2pac,1,2
1,adrenalize,in this moment,0,2
2,animals - original mix,martin garrix,5,2
3,bad medicine,bon jovi,2,2
4,barracuda,heart,0,3
...,...,...,...,...
101,won't get fooled again - original album version,the who,2,2
102,you & me,marc e. bassy,3,2
103,you give love a bad name,bon jovi,16,2
104,you're makin' me high,toni braxton,43,2


In [46]:

# Drop duplicates where have the same track name, track artist and the same track popularity 
df_prep_spotify_song5 = df_prep_spotify_song4.drop_duplicates(subset=['track_name', 'track_artist', 'track_popularity'], keep='first')
print("Number of deleted rows:", len(df_prep_spotify_song4) - len(df_prep_spotify_song5))


Number of deleted rows: 114


In [47]:
# Remove Duplicate Songs Based on the same track_name, track_artist, and duration_ms
# We want to keep only the earliest version of a song (determined by the smallest 'track_album_release_year')
# when there are duplicates in terms of track_name, track_artist, and duration_ms.


# Use groupby on the columns that define a unique song version:
# 'track_name', 'track_artist', and 'duration_ms'. For each group, select the row with the minimum 'track_album_release_year'.
df_prep_spotify_song6 = df_prep_spotify_song5.loc[
    df_prep_spotify_song5.groupby(['track_name', 'track_artist', 'duration_ms'])['track_album_release_year'].idxmin()
].reset_index(drop=True)


# Print the number of rows before this duplicate removal step.
print("Number of rows before removing duplicates by track_name/artist/duration_ms:", len(df_prep_spotify_song5))
# Print the number of rows after duplicate removal and calculate how many rows were deleted.
print("Number of rows after removing duplicates by track_name/artist/duration_ms:", len(df_prep_spotify_song6))
print("Number of deleted rows:", len(df_prep_spotify_song5) - len(df_prep_spotify_song6))


Number of rows before removing duplicates by track_name/artist/duration_ms: 28236
Number of rows after removing duplicates by track_name/artist/duration_ms: 27235
Number of deleted rows: 1001


In [48]:
# Filter all duplicates by 'track_name' and 'track_artist'
duplicates = df_prep_spotify_song6[df_prep_spotify_song6.duplicated(subset=['track_name', 'track_artist'], keep=False)]

# Sort duplicates by the columns you want (e.g., 'track_name' and 'track_artist')
sorted_duplicates = duplicates.sort_values(by=['track_name', 'track_artist'])

# Display the entire data frame of sorted duplicates - we still have about 1991  duplicates
display(sorted_duplicates)

,track_id,track_name,track_artist,track_popularity,track_album_id,track_album_name,playlist_name,playlist_id,playlist_genre,playlist_subgenre,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,track_album_release_year,track_album_release_month,song_age
23,4xkOaSrkexMciUUogZKVTS,'till i collapse,eminem,83,2cWBwpqMsDJC1ZUwz813lo,the eminem show,gangster rap | 100 % gangster,2e0d7otcM1oyecfi3zZPWk,edm,gangster rap,0.548,0.847,1,-3.237,1,0.1860,0.06220,0.000000,0.0816,0.100,171.447,297787,2002,5,23
24,6yr8GiTHWvFfi4o6Q5ebdT,'till i collapse,eminem,16,1ftvBBcu7jYIvXyt3JWB8S,the eminem show,gangster rap,5joQabz9ys3XPGbSx5CaKv,rap,gangster rap,0.551,0.851,1,-3.277,1,0.1970,0.07480,0.000000,0.0784,0.100,171.411,297893,2002,5,23
25,5QTxFnGygVM4jFQiBovmRo,(don't fear) the reaper,blue öyster cult,75,6C9WzlQANeoD0GW5B41YJt,agents of fortune,the sound of album rock,3yj9YnQGTdnFuKbDyXGDi6,rock,album rock,0.333,0.927,9,-8.550,0,0.0733,0.00290,0.000211,0.2970,0.382,141.470,308120,1976,1,49
26,6NTqBHONQqmud0ONBzsLfZ,(don't fear) the reaper,blue öyster cult,7,6YOzCPyuPC92Eg44ag564g,agents of fortune,house of the rising sun,1bMYfBHYBCRHY5LGkjlpSy,rock,album rock,0.368,0.928,9,-8.369,0,0.0593,0.00125,0.001110,0.3010,0.532,141.175,308440,1976,1,49
27,2NL4BBBSgypHnxUKmQdYcT,(don't fear) the reaper,blue öyster cult,10,6NNrQJ8ojvbfFzoUjjABo4,the essential blue öyster cult,supernatural classic rock,6oghIlByD49KFGNmNU8GSH,rock,classic rock,0.366,0.931,0,-8.349,1,0.0609,0.00127,0.002130,0.3050,0.559,141.162,308640,1972,1,53
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27043,0mlAOSILGRys2IbvyOT5VJ,échame la culpa,luis fonsi,75,5C0YLr4OoRGFDaqdMQmkeH,vida,pop latino 2019 - mix canciones reggaeton & pop latino - pop en español 2019 musica para bailar,5ifDekrUt5lbSIMTSzNNqB,latin,latin pop,0.733,0.897,0,-3.652,1,0.0389,0.03910,0.000000,0.1170,0.649,96.006,173800,2019,2,6
27156,4g0KtUbQx4LTGRyPe1AdT9,салам пацанам,stafford63,0,3cQ34CSyMFqjXH1byAZX5L,дым - мой круговорот,russian gangster rap,0u0qLLE8MZc679RZWCk1TT,rap,gangster rap,0.620,0.389,11,-14.509,0,0.2720,0.49300,0.006330,0.1470,0.301,76.105,197538,2018,3,7
27157,7AruZOFWr0uhFNvyCZsmfE,салам пацанам,stafford63,36,1PBTZcxtFSeUBR5ykIEBuh,дым - мой круговорот (special edition),russian gangster rap,0Jw0HckkxCfIrOvpN081eV,rap,gangster rap,0.621,0.392,11,-14.526,0,0.2270,0.48300,0.008650,0.1420,0.325,152.271,197564,2018,3,7
27158,35mUPbEWBUMq6TLl0tkDMk,сарума,kaspiyskiy gruz,27,1XZD2mM4TSMZhvIddq4YBU,треникиспортивки (лучшее),russian gangster rap,0u0qLLE8MZc679RZWCk1TT,rap,gangster rap,0.651,0.775,6,-7.099,0,0.1790,0.42600,0.000012,0.2210,0.567,81.961,142316,2014,4,11


In [49]:
# Remove Duplicate Songs Based on track_name, track_artist, and the First 3 Digits of Tempo
# While keeping the earliest version by track_album_release_year and track_album_release_month.

# Create a new column 'tempo_first3' representing the integer part of the tempo.
# This effectively captures the first 3 digits (if tempo is in the range 100-999).
df_prep_spotify_song6['tempo_first3'] = df_prep_spotify_song6['tempo'].astype(int)

# Group by the key columns: 'track_name', 'track_artist', and 'tempo_first3'.
# For each group, sort by 'track_album_release_year' and 'track_album_release_month' (ascending)
# and select the index of the first row (i.e., the earliest version).
idx = df_prep_spotify_song6.groupby(['track_name', 'track_artist', 'tempo_first3']).apply(
    lambda group: group.sort_values(['track_album_release_year', 'track_album_release_month']).index[0]
).values


# Create a new DataFrame keeping only the selected rows.
df_prep_spotify_song7 = df_prep_spotify_song6.loc[idx].reset_index(drop=True)

df_prep_spotify_song7 = df_prep_spotify_song7.drop(columns=['tempo_first3'])

# Print the number of rows and deleted rows after duplicate removal.
print("Number of rows after removal by track_name/artist/tempo:", len(df_prep_spotify_song7))
print("Number of duplicate deleted rows:", len(df_prep_spotify_song6) - len(df_prep_spotify_song7))


Number of rows after removal by track_name/artist/tempo: 26420
Number of duplicate deleted rows: 815


In [50]:
# Filter all duplicates by 'track_name' and 'track_artist'
duplicates = df_prep_spotify_song7[df_prep_spotify_song7.duplicated(subset=['track_name', 'track_artist'], keep=False)]

# Sort duplicates by the columns you want (e.g., 'track_name' and 'track_artist')
sorted_duplicates = duplicates.sort_values(by=['track_name', 'track_artist'])

print(f"{len(df_prep_spotify_song7)} rows in the data")

# Display the entire DataFrame of sorted duplicates 
display(sorted_duplicates) 

26420 rows in the data


,track_id,track_name,track_artist,track_popularity,track_album_id,track_album_name,playlist_name,playlist_id,playlist_genre,playlist_subgenre,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,track_album_release_year,track_album_release_month,song_age
30,5FKkbyGm42hw1y08l2knlO,(i can't get no) satisfaction - mono version,the rolling stones,56,0aqZJlugIkTUWW1sa4BANp,hot rocks (1964-1971),80s / classic rock,29dTrOurPDrMcrnio2q6hZ,rock,classic rock,0.731,0.837,9,-8.673,1,0.0477,0.000287,0.150000,0.0978,0.832,135.603,222747,1971,12,54
31,2PzU4IB8Dr6mxV3lHuaG34,(i can't get no) satisfaction - mono version,the rolling stones,76,2Q5MwpTmtjscaS34mJFXQQ,out of our heads,house of the rising sun,1bMYfBHYBCRHY5LGkjlpSy,rock,album rock,0.723,0.863,2,-7.890,1,0.0338,0.038300,0.031700,0.1280,0.931,136.302,222813,1965,7,60
32,1RvuWdj9YsXn4cMMLXr85Z,(i can't get no) satisfaction - mono version / remastered 2002,the rolling stones,0,2aEyBG1K73KtNscS50y4aO,hot rocks 1964-1971,classic hard rock,1gAeQGQ7hr2q7IYmaejDW6,rock,hard rock,0.723,0.841,9,-8.649,1,0.0481,0.000324,0.174000,0.1020,0.854,135.595,222800,1971,12,54
33,57J2znxukXsXzS3XPuZ1TG,(i can't get no) satisfaction - mono version / remastered 2002,the rolling stones,8,1YYEC5hLwDbQPtErSBD223,the rolling stones singles collection: the london years (remastered),workday: rock classics,37i9dQZF1DX1lwxXvReaux,rock,classic rock,0.722,0.882,2,-6.763,1,0.0348,0.035400,0.049600,0.1190,0.921,136.299,222840,1989,8,36
174,6ce41JVa6KFaryfmC2bu2W,22,taylor swift,68,4jTYApZPMapg56gRycOn0D,red (big machine radio release special),post teen pop,6rjxP7GQKoqqgoakzxl3PY,pop,post-teen pop,0.661,0.729,7,-6.561,1,0.0376,0.002150,0.001300,0.0477,0.668,103.987,232120,2012,10,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26099,7hrDA2n8BVGLyHra0rBiFd,younger - kygo remix,seinabo sey,57,7jzURx17ZW6Yo5153MrLdh,younger (the remixes),edm tropical,3pS63EDS40FVGYL41zAcU4,latin,tropical,0.690,0.489,5,-7.938,1,0.0788,0.130000,0.000220,0.0815,0.645,114.008,351573,2014,1,11
26104,6PPtwNxnMMfcWexUVZ8nYp,your body's callin',r. kelly,0,5nUPkSF3RI2Ii7ZwXeOk61,12 play,"back in the day - r&b, new jack swing, swingbeat, rnb..etc.",3a9y4eeCJRmG9p4YKfqYIx,r&b,new jack swing,0.702,0.533,9,-7.634,1,0.0284,0.451000,0.000701,0.1060,0.720,84.151,276147,1993,1,32
26105,4iXZFxEUzTTLMLE5jCyKEb,your body's callin',r. kelly,48,11UWeyxXNreHeZwEViqs3y,12 play,new jack swing/ r&b hits: 1987 - 2002,4sji14lrB5bgcr51lPALYH,r&b,new jack swing,0.658,0.550,9,-6.705,1,0.0302,0.457000,0.000728,0.1140,0.727,168.289,277493,1993,11,32
26342,4g0KtUbQx4LTGRyPe1AdT9,салам пацанам,stafford63,0,3cQ34CSyMFqjXH1byAZX5L,дым - мой круговорот,russian gangster rap,0u0qLLE8MZc679RZWCk1TT,rap,gangster rap,0.620,0.389,11,-14.509,0,0.2720,0.493000,0.006330,0.1470,0.301,76.105,197538,2018,3,7


In [51]:
print(f"We last with 513 songs that have the same artist name and the same song name. This accounts for {round(513 / len(df_prep_spotify_song7) * 100, 2)}% of the data.")
print("There is no need to delete more duplicates as some songs can be in different slightly different versions") 

We last with 513 songs that have the same artist name and the same song name. This accounts for 1.94% of the data.
There is no need to delete more duplicates as some songs can be in different slightly different versions


In [52]:
# Convert specific columns to int64
df_prep_spotify_song7['track_album_release_year'] = df_prep_spotify_song7['track_album_release_year'].astype('int64')
df_prep_spotify_song7['track_album_release_month'] = df_prep_spotify_song7['track_album_release_month'].astype('int64')
df_prep_spotify_song7['song_age'] = df_prep_spotify_song7['song_age'].astype('int64')

In [69]:
# Check how many duplicates (same song name and same artist name) have the popularity of zero. 

# Filter to get only groups (same track_name and track_artist) that are duplicates (more than one row)
duplicate_entries = df_prep_spotify_song7.groupby(['track_name', 'track_artist']).filter(lambda x: len(x) > 1)

# Count how many of these duplicate rows have track_popularity == 0
zero_popularity_count = duplicate_entries[duplicate_entries['track_popularity'] == 0].shape[0]

# Total number of duplicate rows
total_duplicate_rows = duplicate_entries.shape[0]

# Print results
print("Number of duplicate rows (same track_name & track_artist) with popularity=0:", zero_popularity_count)
print("Percentage of duplicate rows with popularity=0: {:.2f}%".format(zero_popularity_count / total_duplicate_rows * 100))

Number of duplicate rows (same track_name & track_artist) with popularity=0: 61
Percentage of duplicate rows with popularity=0: 11.89%


In [53]:
df_prep_spotify_song7.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26420 entries, 0 to 26419
Data columns (total 25 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   track_id                   26420 non-null  string 
 1   track_name                 26420 non-null  string 
 2   track_artist               26420 non-null  string 
 3   track_popularity           26420 non-null  int64  
 4   track_album_id             26420 non-null  string 
 5   track_album_name           26420 non-null  string 
 6   playlist_name              26420 non-null  string 
 7   playlist_id                26420 non-null  string 
 8   playlist_genre             26420 non-null  string 
 9   playlist_subgenre          26420 non-null  string 
 10  danceability               26420 non-null  float64
 11  energy                     26420 non-null  float64
 12  key                        26420 non-null  int64  
 13  loudness                   26420 non-null  flo

In [54]:
# Count how many categories are in the 'subgenre' column
df_prep_spotify_song7['playlist_subgenre'].nunique() # 24 subgenres


24

# Data Consistency and Integrity Checks

In [60]:
# We'll print summary statistics for numeric columns to verify their ranges and distributions.

print("Summary statistics for numeric columns:")
display(df_prep_spotify_song7.describe())

# You might also want to check for any unexpected missing values in the cleaned data:
print("\nMissing values by column:")
display(df_prep_spotify_song7.isnull().sum())

Summary statistics for numeric columns:


,track_popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,track_album_release_year,track_album_release_month,song_age
count,26420.000000,26420.000000,26420.000000,26420.000000,26420.000000,26420.000000,26420.000000,26420.000000,26420.000000,26420.000000,26420.000000,26420.000000,26420.000000,26420.000000,26420.000000,26420.000000
mean,39.366730,0.653355,0.696305,5.375814,-6.845867,0.564572,0.108797,0.180718,0.094890,0.191244,0.506827,121.029739,225684.974603,2011.341181,6.149962,13.658819
std,23.281363,0.145818,0.184764,3.614341,3.055129,0.495822,0.103357,0.225898,0.237269,0.156650,0.234649,27.007879,61349.263642,11.134976,3.830870,11.134976
min,0.000000,0.077100,0.000175,0.000000,-46.448000,0.000000,0.022400,0.000001,0.000000,0.009360,0.000010,35.477000,31429.000000,1957.000000,1.000000,5.000000
25%,22.000000,0.561000,0.576000,2.000000,-8.344000,0.000000,0.041000,0.014400,0.000000,0.092800,0.325000,99.971750,186826.000000,2008.000000,2.000000,6.000000
50%,42.000000,0.670000,0.720000,6.000000,-6.284000,1.000000,0.063100,0.081700,0.000022,0.127000,0.508000,122.012000,215869.500000,2016.000000,6.000000,9.000000
75%,57.000000,0.760000,0.842000,9.000000,-4.725000,1.000000,0.135000,0.267000,0.007443,0.248000,0.691000,134.070500,254083.250000,2019.000000,10.000000,17.000000
max,98.000000,0.983000,1.000000,11.000000,1.275000,1.000000,0.918000,0.994000,0.994000,0.996000,0.990000,239.440000,517810.000000,2020.000000,12.000000,68.000000



Missing values by column:


track_id                     0
track_name                   0
track_artist                 0
track_popularity             0
track_album_id               0
track_album_name             0
playlist_name                0
playlist_id                  0
playlist_genre               0
playlist_subgenre            0
danceability                 0
energy                       0
key                          0
loudness                     0
mode                         0
speechiness                  0
acousticness                 0
instrumentalness             0
liveness                     0
valence                      0
tempo                        0
duration_ms                  0
track_album_release_year     0
track_album_release_month    0
song_age                     0
dtype: int64

In [62]:
# save a copy before moving further
# Copying all string columns
spotify_string_column_copy = df_prep_spotify_song7.select_dtypes(include='object').copy()

# Copying all numeric columns (e.g., int or float)
spotify_numeric_columns_copy = df_prep_spotify_song7.select_dtypes(include=['int64', 'float64']).copy()

# Using pickle to save the data until this step for EDA step

In [65]:
df_prep_spotify_song7.to_pickle(r'C:\Users\kagan\Dropbox\קורס DS בר-אילן\שיעורים\Projects\Project 3 - ML preparation and model\Spotify_data\flat_file_end_pre_step.pkl')


Number of duplicate rows (same track_name & track_artist) with popularity=0: 61
Percentage of duplicate rows with popularity=0: 11.89%
